In [0]:
# dbutils.library.restartPython()

# ⬇️ 1. Clean up sys.path – ONE project root only
import sys, importlib, pathlib

root = pathlib.Path("/Workspace/Users/tom_corbin1@hotmail.com/ETL-framework")

# Drop any entry that lives inside the src tree
sys.path = [p for p in sys.path if not p.endswith(("src", "src/delta_engine"))]

# Put the project root right at the front
sys.path.insert(0, str(root))

# ⬇️ 2. Nuke the negative import cache
importlib.invalidate_caches()


from pyspark.sql import SparkSession
import pyspark.sql.types as T

from src.delta_engine.engine import Engine, OrchestratorOptions
from src.delta_engine.state.ports import Aspect, SnapshotPolicy
from src.delta_engine.models import Table, Column
from src.delta_engine.desired.builders import build_desired_catalog

In [0]:
tables = [
    Table(
        catalog_name="dev",
        schema_name="silver",
        table_name="demo_table",
        columns=[
            Column("id", T.IntegerType(), is_nullable=False, comment="pk"),
            Column("name", T.StringType(), is_nullable=True, comment="name"),
        ],
        comment="Demo table",
        properties={"delta.columnMapping.mode": "name"},
        primary_key=["id"],
    )
]

options = OrchestratorOptions(
    aspects=frozenset({Aspect.SCHEMA, Aspect.COMMENTS, Aspect.PROPERTIES, Aspect.PRIMARY_KEY}),
    snapshot_policy=SnapshotPolicy.PERMISSIVE,
    execute=True,
    fail_on_validation_errors=True,
)

desired_catalog = build_desired_catalog(tables)


engine = Engine(spark)
report = engine.run(desired_catalog, options)
print("Actions:", len(report.plan.actions))
print("Validation OK?", report.validation.ok)
